In [116]:
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
from pprint import pprint
import time

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}

browser = Browser('chrome', **executable_path, headless=False)

In [4]:
url = 'https://www.appannie.com/dashboard/home'
browser.visit(url)

In [5]:
browser.find_by_name(name = 'username').type('devindubin@gmail.com')

'devindubin@gmail.com'

In [6]:
password = '4rFR!gPHpY4tU8y'

In [7]:
browser.find_by_css('input')[1].type(password)


'4rFR!gPHpY4tU8y'

In [8]:
browser.find_by_tag('button').click()

In [9]:
browser.find_by_text('Top Charts').click()

In [10]:
from datetime import datetime, timedelta

In [173]:
today = datetime.now()
#.strftime("%Y-%m-%d")
dates = [(today - timedelta(days=i)).strftime("%Y-%m-%d") for i in range(90)]

In [174]:
#https://www.appannie.com/apps/ios/top-chart?country=US&category=36&device=iphone&date=2020-12-01&feed=All&page_number=0&page_size=500&rank_sorting_type=rank
list_of_builds = []

for moment in dates:
    url = f'https://www.appannie.com/apps/ios/top-chart?country=US&category=36&device=iphone&date={moment}&feed=All&page_number=0&page_size=500&rank_sorting_type=rank'
    browser.visit(url)
    time.sleep(2)
    html = browser.html
    soup = BeautifulSoup(html,'html.parser')
    all_trs = soup.table.tbody.find_all('tr')
    build_df = {'Rank':[],'Date': moment,'Free':[],'Paid':[],'Grossing':[]}
    for _ in all_trs:
        current_rank_apps = [elem.text for elem in _.select('a span')]

        rank_num = _.span.text
        build_df['Rank'].append((rank_num))
        build_df['Free'].append([current_rank_apps[0],current_rank_apps[1]])
        build_df['Paid'].append([current_rank_apps[2],current_rank_apps[3]])
        build_df['Grossing'].append([current_rank_apps[4],current_rank_apps[5]])
    
    list_of_builds.append(build_df)

    

AttributeError: 'NoneType' object has no attribute 'tbody'

In [175]:
builds_to_concat = [pd.DataFrame(_).set_index('Date') for _ in list_of_builds]
pd.concat(builds_to_concat)

,Rank,Free,Paid,Grossing
Date,,,,
2020-12-01,1,"[Roof Rails, Voodoo]","[Minecraft, Mojang]","[YouTube: Watch, Listen, Stream, Google]"
2020-12-01,2,"[ZOOM Cloud Meetings, Zoom Video Communications]","[Bloons TD 6, Ninja Kiwi]","[Clash of Clans, Supercell]"
2020-12-01,3,"[YouTube: Watch, Listen, Stream, Google]","[Heads Up! - Trivia on the go, Warner Bros]","[Disney+, Disney]"
2020-12-01,4,"[Among Us!, InnerSloth]","[Monopoly, Marmalade]","[Tinder - Dating & Make Friends, Match Group]"
2020-12-01,5,"[TikTok, TikTok Pte Ltd]","[Procreate Pocket, Savage Interactive]","[Roblox, Roblox]"
...,...,...,...,...
2020-11-19,496,"[healow, eClinicalWorks]","[PDG PROmote 2019-2021, ForceReadiness.com]","[DAZN: Live Boxing & MMA, DAZN]"
2020-11-19,497,"[Current - Bank for Modern Life, Finco Services]","[NeuralCam NightMode, Halcyon Products]","[Torch-Meet new people, Torch]"
2020-11-19,498,"[・MEGA・, Mega Limited]","[Spectre Camera, Chroma Noir]","[Mahjong+, Jam City]"


In [171]:
url = f'https://www.appannie.com/apps/ios/top-chart?country=US&category=36&device=iphone&date={moment}&feed=All&page_number=0&page_size=500&rank_sorting_type=rank'
browser.visit(url)

'Roof Rails'

['Roof Rails',
 'Voodoo',
 'Minecraft',
 'Mojang',
 'YouTube: Watch, Listen, Stream',
 'Google']

,#,Free,Paid,Grossing
,#,Free,Paid,Grossing
0,1,Threads from Instagram Instagram =,Minecraft Mojang = $,Roblox Roblox = $
1,2,Walmart - Shopping & Grocery Walmart ▲ 14,Heads Up! - Trivia on the go Warner Bros = $,"YouTube: Watch, Listen, Stream Google ▲ 1 $"
2,3,Among Us! InnerSloth = $,Human Anatomy Atlas 2021 Visible Body ▲ 2 $,Disney+ Disney ▼ 1 $
3,4,Roof Rails Voodoo = $,Monopoly Marmalade = $,Call of Duty®: Mobile Activision Publishing ...
4,5,Project Makeover Bubblegum Games ▼ 3 $,Bloons TD 6 Ninja Kiwi ▲ 2 $,Tinder - Dating & Make Friends Match Group ▼...
...,...,...,...,...
495,496,Jetpack Joyride Halfbrick ▲ 210 $,Rhythm Sight Reading Trainer Rolfs Apps ▲ 108,Rival Stars Horse Racing PikPok ▲ 18 $
496,497,Sephora: Top Makeup & Skincare Sephora ▼ 39,Pill Identifier by Drugs.com Drugs ▲ 323,Followers Pro + Dynamic Slash LLC ▼ 11 $
497,498,fuboTV: Watch Live Sports & TV fuboTV ▲ 357,Stamena - Longer lasting sex Nathaniel Eliaso...,5 Differences Online Smart Project ▲ 2 $


1